In [1]:
import openpyxl
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [2]:
## # USUNIECIE BAZY STOLARNIA
## from sqlite3 import connect
## cnn = connect("drive/MyDrive/GENERATORY.db")
## cnn.execute("DROP TABLE baza_STOLARNIA")
## cnn.commit()
## cnn.close()
generatory_path = "Z:/450. PLANISTA - ZAOPATRZENIE/GENERATORY.db"
# # USUNIECIE WSZYTKICH WIERSZY Z BAZY STOLARNIA
from sqlite3 import connect
cnn = connect(generatory_path)
cnn.execute("DELETE FROM baza_STOLARNIA")
cnn.commit()
cnn.close()

In [3]:
cgb = openpyxl.load_workbook("Z:/450. PLANISTA - ZAOPATRZENIE/CZASY_baza_2413.xlsx")
technologia = cgb["DATABASE"]

POZ = list()

# DRE
for r in technologia["A2:J11383"]:
  POZ.append([x.value for x in r])

# PWI
for r in technologia["K2:T9262"]:
  POZ.append([x.value for x in r])

# PPI
for r in technologia["U2:AD4675"]:
  POZ.append([x.value for x in r])

# SKL
for r in technologia["AE2:AN814"]:
  POZ.append([x.value for x in r])

# KRT
for r in technologia["AO2:AX1150"]:
  POZ.append([x.value for x in r])

#skl2
for r in technologia["AY2:BH5189"]:
  POZ.append([x.value for x in r])

# plw
for r in technologia["BI2:BR108"]:
  POZ.append([x.value for x in r])

#skl3
for r in technologia["BS2:CB980"]:
  POZ.append([x.value for x in r])

# plp
for r in technologia["CC2:CL74"]:
  POZ.append([x.value for x in r])

# mdf
for r in technologia["CM2:CV84"]:
  POZ.append([x.value for x in r])

In [4]:
columns = ("MODEL", "bryla", "MATERIAL", "czas", "numer", "wymiar", "ilosc", "oznacz", "link", "format")
_poz = pd.DataFrame(POZ, columns=columns)

In [5]:
poz = _poz[~_poz.MODEL.isna()]


In [6]:
valid_models = ['AVANT','DU2 wersja II', 'SAMOA',
       'ELIXIR', 'MYSTIC', 'OVAL',
        'HORIZON', 'RITZ', 'ONYX', 'DIVA',
       'SPECTRA', 'LENOX',  'STONE',
       'HUDSON', 'WILLOW', 'LOBBY', 'EXTREME', 'CALYPSO', 'REVERSO',
       'AMALFI', 'KELLY', 'UNO', 'OXYGEN']

poz = poz[poz.MODEL.str.contains("|".join(valid_models))]

In [7]:
poz

,MODEL,bryla,MATERIAL,czas,numer,wymiar,ilosc,oznacz,link,format
18,AVANT,1,Sosna,18,2344,25x50x828,2,OP,None,None
19,AVANT,1.5,Sosna,18,2344,25x50x828,2,OP,None,None
20,AVANT,2,Sosna,18,2344,25x50x828,2,OP,None,None
21,AVANT,2.5,Sosna,18,2344,25x50x828,2,OP,None,None
22,AVANT,3,Sosna,18,2344,25x50x828,2,OP,None,None
...,...,...,...,...,...,...,...,...,...,...
33648,MYSTIC,"D2,5N",MDF 8mm,0,0002,50x150,4,None,None,None
33649,MYSTIC,D3N,MDF 8mm,0,0002,50x150,4,None,None,None
33650,MYSTIC,"D3,5N",MDF 8mm,0,0002,50x150,4,None,None,None
33707,ONYX,NLEWY,MDF 8mm,0,0003,27x40,2,PP,None,None


In [8]:
#@title Cleaning data

def wymiary(wymiar, format):
  if format == None:
    return wymiar
  elif format != None:
    return "*** " + format

def pars_str(x):
  if type(x) == str:
    try:
      return np.int32(x)
    except:
      return np.int32(0)
  else:
    return np.int32(x)

def pars_float(x):
  if type(x) == float:
    px = str(x)
    if px[-1] == "0":
      return px[0]
    else:
      return px.replace(".", ",")
  return x

def pars_float2(x):
  if type(x) == float:
    return str(x)[:-2]
  return x

def oznaczenia(x):
  if type(x) == str:
    return x.strip()
  elif type(x) == float:
    return str(x)
  elif x == None:
    return "brak"

def wys(mat, wym):
  typ = {'Płyta wiórowa 16mm': 15, 'Płyta pilśniowa 3mm': 3,
       'Sklejka 15mm': 15,  'Sklejka 18mm': 18,
        'Sklejka 24mm': 24}
  if mat in list(typ.keys()):
    return typ[mat]
  elif len(wym.split("x")) > 2:
    return int(wym.split("x")[0].replace("*** ", ""))

  return 0

def szer(s):
  try:
    return int(s.lower().split("x")[-2].replace("*** ", ""))
  except:
    return None

def dlug(d):
  try:
    return int(d.lower().split("x")[-1])
  except:
    return None

poz["WYMIAR"] = poz.apply(lambda x: wymiary(x.wymiar,x.format), axis=1)
poz["_CZAS"] = poz["czas"].apply(pars_str)
poz["_ILOSC"] = poz["ilosc"].apply(pars_str)
poz["BRYLA"] = poz["bryla"].apply(pars_float)
poz["NUMER"] = poz["numer"].apply(pars_float2)
poz["OZNACZ"] = poz["oznacz"].apply(oznaczenia)
poz["WYS"] = poz.apply(lambda x: wys(x.MATERIAL, x.WYMIAR), axis=1)
poz["SZER"] = poz.WYMIAR.apply(szer)
poz["DLUG"] = poz.WYMIAR.apply(dlug)
# poz[["WYS", "SZER", "DLUG"]] = poz.WYMIAR.apply(pars_wymiary)

poz.drop(["numer", "bryla", "czas", "ilosc", "wymiar", "format", "link", "oznacz"], axis=1, inplace=True)

In [9]:
poz

,MODEL,MATERIAL,WYMIAR,_CZAS,_ILOSC,BRYLA,NUMER,OZNACZ,WYS,SZER,DLUG
18,AVANT,Sosna,25x50x828,18,2,1,2344,OP,25,50.0,828.0
19,AVANT,Sosna,25x50x828,18,2,"1,5",2344,OP,25,50.0,828.0
20,AVANT,Sosna,25x50x828,18,2,2,2344,OP,25,50.0,828.0
21,AVANT,Sosna,25x50x828,18,2,"2,5",2344,OP,25,50.0,828.0
22,AVANT,Sosna,25x50x828,18,2,3,2344,OP,25,50.0,828.0
...,...,...,...,...,...,...,...,...,...,...,...
33648,MYSTIC,MDF 8mm,50x150,0,4,"D2,5N",0002,brak,0,50.0,150.0
33649,MYSTIC,MDF 8mm,50x150,0,4,D3N,0002,brak,0,50.0,150.0
33650,MYSTIC,MDF 8mm,50x150,0,4,"D3,5N",0002,brak,0,50.0,150.0
33707,ONYX,MDF 8mm,27x40,0,2,NLEWY,0003,PP,0,27.0,40.0


In [10]:
#@title Zapis do bazy danych
from sqlalchemy import create_engine

engine = create_engine("sqlite:///"+generatory_path, echo=False)
poz.to_sql("baza_STOLARNIA", con=engine, if_exists="append", index=False)

7028